In [1]:
from test import *

BGM_data = get_BGM_data("날림v")

START crawling
FIND 18 lists in 날림v CHANNEL
END crawling
GET BGM Data from 날림v CHANNEL in 58.46


In [2]:
total_BGM = [y for x in BGM_data.values() for y in x['items']]
print(len(total_BGM))
total_BGM = list(dict.fromkeys(total_BGM))
print(len(total_BGM))

1960
1207


![](youtube_playlist.png)

In [ ]:
import os
import pymysql
from dotenv import load_dotenv

load_dotenv()
DATABASE_HOST = os.getenv('DATABASE_HOST')
DATABASE_PORT = int(os.getenv('DATABASE_PORT'))
DATABASE_USER = os.getenv('DATABASE_USER')
DATABASE_PASSWORD = os.getenv('DATABASE_PASSWORD')
db = pymysql.connect(
        host=DATABASE_HOST,
        port=DATABASE_PORT,
        user=DATABASE_USER,
        passwd=DATABASE_PASSWORD,
        charset='utf8mb4'
)
cursor = db.cursor()

with open("schema.sql", 'r') as f:
    sqls = [x for x in f.read().split(";") if x != '']

for sql in sqls:
    cursor.execute(sql)
db.commit()
db.close()

In [ ]:
# SELECT name FROM done;
# INSERT INTO users (name) VALUES ('alskfl');
# SELECT name, uploader, href FROM songs
# INSERT INTO songs (name, uploader, href) VALUES ("{row[0]}", "{row[1]}", "{row[2]}")
# INSERT INTO done (name) VALUES ("{year}-{month}")